In [5]:
import os
import sys
import torch
import numpy as np
import SimpleITK as sitk

from PIL import Image

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from pyradiomics_api import pyradiomics_glcm
from diffglcm import DiffGLCM

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Cameraman.png and convert to numpy array with values in range 0-1
img = Image.open(r'../../pyradiomics/Cameraman.png').convert('L')  # Convert to grayscale
img = img.resize((256, 256))
image_array = np.array(img, dtype=np.float32) / 255.0  # Normalize to 0-1 range
image_tensor = torch.from_numpy(image_array).to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, H, W]
image_tensor = image_tensor

# == Compute GLCM features Using Pyradiomics ==
pyradiomics_glcm, pyradiomics_glcm_features = pyradiomics_glcm(image_array, low_bound=0, high_bound=1, bin_count=16, symmetricalGLCM=True, angle=(1,0))
# print(pyradiomics_glcm_features.shape)  

# == Compute GLCM features Using DiffGLCM ==

# Initialize DiffGLCM
diffglcm = DiffGLCM(
    image_size=image_array.shape[0],  # Assuming square image
    low_bound=0,
    high_bound=1,
    Ng=16,
    alpha=10,
    differentiable=True
).to(device)

# Compute GLCM and features
with torch.no_grad():  # Disable gradient computation for inference
    diffrad_glcm, diffrad_features = diffglcm(image_tensor, offset_r=1, offset_c=0)

# Move features back to CPU for printing
diffrad_features = diffrad_features.cpu()

# == Compare Pyradiomics and DiffGLCM features ==
feature_names = [
    'Autocorrelation', 'ClusterProminence', 'ClusterShade', 'ClusterTendency',
    'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy',
    'DifferenceVariance', 'Id', 'Idm', 'Idmn', 'Idn', 'Imc1', 'Imc2',
    'InverseVariance', 'JointAverage', 'JointEnergy', 'JointEntropy',
    'MCC', 'MaximumProbability', 'SumAverage', 'SumEntropy', 'SumSquares'
]

print(f"{'Feature Name':<30} | {'DiffGLCM':<30} | {'Pyradiomics GLCM':<30}")
print("-" * 93)  # Print a separator line
for i, name in enumerate(feature_names):
    diff_val = diffrad_features[0, i, 0].item()
    pyrad_val = pyradiomics_glcm_features[name][0]
    print(f"{name:<30} | {diff_val:<30} | {pyrad_val:<30}")



GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Feature Name                   | DiffGLCM                       | Pyradiomics GLCM              
---------------------------------------------------------------------------------------------
Autocorrelation                | 123.14684295654297             | 123.1468443627451             
ClusterProminence              | 19325.39453125                 | 19325.394929071015            
ClusterShade                   | -266.71240234375               | -266.7126489747228            
ClusterTendency                | 103.61572265625                | 103.61571146553271            
Contrast                       | 2.0674479007720947             | 2.0674479166666666            
Correlation                    | 0.9608777761459351             | 0.9608746004802938            
DifferenceAverage              | 0.5344209671020508             | 0.5344209558823529            
DifferenceEntropy              | 1.334649920463562              | 1.3346498280316017            
DifferenceVariance             | 